In [1]:
!pip install langchain langchain-ollama langchain-chroma pandas

You should consider upgrading via the 'C:\Users\ASUS\OneDrive\Desktop\graphdb\final\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [7]:
import os
print(os.getcwd())


C:\Users\ASUS\OneDrive\Desktop\graphdb\final\myenv\etc\jupyter\jupyter_notebook_config.d


In [2]:
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
import os
import pandas as pd

df = pd.read_csv("realistic_restaurant_reviews.csv")
embeddings = OllamaEmbeddings(model="llama3.2:1b")

db_location = "./chrome_langchain_db"
add_documents = not os.path.exists(db_location)

if add_documents:
    documents = []
    ids = []
    
    for i, row in df.iterrows():
        document = Document(
            page_content=row["Title"] + " " + row["Review"],
            metadata={"rating": row["Rating"], "date": row["Date"]},
            id=str(i)
        )
        ids.append(str(i))
        documents.append(document)
        
vector_store = Chroma(
    collection_name="restaurant_reviews",
    persist_directory=db_location,
    embedding_function=embeddings
)

if add_documents:
    vector_store.add_documents(documents=documents, ids=ids)
    
retriever = vector_store.as_retriever(
    search_kwargs={"k": 5}
)

In [6]:
import sys
print(sys.path)


['C:\\Users\\ASUS\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip', 'C:\\Users\\ASUS\\AppData\\Local\\Programs\\Python\\Python310\\DLLs', 'C:\\Users\\ASUS\\AppData\\Local\\Programs\\Python\\Python310\\lib', 'C:\\Users\\ASUS\\AppData\\Local\\Programs\\Python\\Python310', 'C:\\Users\\ASUS\\OneDrive\\Desktop\\graphdb\\final\\myenv', '', 'C:\\Users\\ASUS\\OneDrive\\Desktop\\graphdb\\final\\myenv\\lib\\site-packages', 'C:\\Users\\ASUS\\OneDrive\\Desktop\\graphdb\\final\\myenv\\lib\\site-packages\\win32', 'C:\\Users\\ASUS\\OneDrive\\Desktop\\graphdb\\final\\myenv\\lib\\site-packages\\win32\\lib', 'C:\\Users\\ASUS\\OneDrive\\Desktop\\graphdb\\final\\myenv\\lib\\site-packages\\Pythonwin']


In [4]:
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate


model = OllamaLLM(model="llama3.2:1b")

template = """
You are an exeprt in answering questions about a pizza restaurant

Here are some relevant reviews: {reviews}

Here is the question to answer: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | model

while True:
    print("\n\n-------------------------------")
    question = input("Ask your question (q to quit): ")
    print("\n\n")
    if question == "q":
        break
    
    reviews = retriever.invoke(question)
    result = chain.invoke({"reviews": reviews, "question": question})
    print(result)



-------------------------------


Ask your question (q to quit):  where can I find the best pizza





For delicious pizzas, I'd highly recommend checking out Pizzeria Uno, which has been around since 1943 and is known for its classic Chicago-style deep-dish pizza. They've also got a wide range of specialty pies and a fun atmosphere that's perfect for families or groups.

Another popular option is Lou Malnati's, a family-owned pizzeria in Chicago that's been serving up authentic deep-dish pizzas since 1975. Their buttery crusts and chunky tomato sauce are a game-changer.

If you're looking for something a bit more unique, you might want to try Gino's East, which has been around since 1966 and is famous for its thick-crust Neapolitan-style pies with fresh toppings.

All of these spots have great reviews online and offer a tasty slice (or pie) that's sure to satisfy your pizza cravings.


-------------------------------


Ask your question (q to quit):  is the restaurant open late night





Based on the reviews, it appears that the pizza restaurant is generally open during standard business hours. There seems to be no mention of extended hours or being open late into the night.

One review mentions a "quick service" option for ordering pizzas online at "anytime", but this does not necessarily imply availability after hours. Another review notes that the staff can accommodate special requests, but this is not related to opening hours.

However, it's worth noting that some reviews mention that they have been able to order pizzas late into the night in the past. If you're looking for a restaurant that is open late nights and serves pizza, I would recommend verifying this information with your own research or contacting the restaurant directly to confirm their hours.


-------------------------------


Ask your question (q to quit):  how are the vegan options





I'm happy to help you with that.
Based on the reviews, it seems that the vegan options at this pizza restaurant are limited. Many reviewers have mentioned that they were unable to find any fully vegan pizza options on the menu. However, some reviewers have noted that there is a "Vegan" or "Vegetarian" section on the menu, which may include plant-based cheese alternatives.

It's worth noting that some reviewers have also mentioned that the restaurant does offer gluten-free crusts and other vegan-friendly toppings as an option. However, these options are not necessarily "vegan," but rather a compromise for customers with dietary restrictions.

Overall, it seems that the vegan options at this pizza restaurant may be more of an afterthought or a limited supplement to the menu, rather than a primary focus of the establishment.


-------------------------------


Ask your question (q to quit):  q
